# iQ Winter School 2018 on Machine Learning Applied to Quantitative Analysis of Medical Images
## Hands-on Session 2 - Challenge

### Introduction: unsupervised image segmentation example

In this notebook, you can follow the steps of an image segmentation method that uses basic image processing techniques. As you will see, the image labels are not used for developing this method.

This gives you an idea of how well such a simplistic approach performs on this dataset.

In [ ]:
import numpy as np
from pylab import *
import skimage.io, skimage.filters, skimage.exposure, skimage.segmentation
import scipy.ndimage as sndim
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

# Loads the validation data and shows one example
data_validation = np.load(r'data.npy')
labels_validation = np.load(r'labels.npy')
n_images = data_validation.shape[0]

img_ind = 0
img = data_validation[img_ind,...]
ground_truth = labels_validation[img_ind,...]

contours_ground_truth = skimage.segmentation.mark_boundaries(img,ground_truth)
figure(figsize=(20,20))
subplot(131)
imshow(img, cmap='gray')
title('original image', fontsize=18)
axis('off')
subplot(132)
imshow(ground_truth, cmap='gray')
title('ground truth', fontsize=18)
axis('off')
subplot(133)
imshow(contours_ground_truth)
title('contour overlay', fontsize=18)
axis('off')
show()

We can see that the cell membranes have lower intensities than the cell body, so we can first try a number of simple thresholding methods on the pixel intensities.

### Simple thresholding

In [ ]:
# Smooths the image with a gaussian filter
smoothed_img = sndim.gaussian_filter(img, sigma=2)

# Applies a number of thresholding techniques
skimage.filters.try_all_threshold(smoothed_img, figsize=(14,14))
show()

# Takes the threshold determined by the "threshold_minimum" method and binarizes the image
th = skimage.filters.thresholding.threshold_minimum(smoothed_img)
min_bin = smoothed_img > th

The "minimum threshold" gives a cleaner segmentation than the other methods. However, and although we can already obtain a first approximation of the desired segmentation, we are excluding areas within the cell bodies that have intensities in the same range as the membranes.

Another option is to use a filter than enhances tubular structures. We can then segment the result to obtain a binary segmentation and apply some morphological operations to post-process this segmentation.

### "Vesselness" filter

In [ ]:
# Applies Frangi's vesselness filter
vesselness = skimage.filters.frangi(img, scale_range=(3,5))
vesselness /= vesselness.max()

# Binarizes the vesselness
ves_bin = vesselness <.1

# Removes small objects
l, n = sndim.label(1-ves_bin)
sizes = np.bincount(l.ravel())
mask_sizes = sizes > 100
mask_sizes[0] = 0
ves_bin = 1-mask_sizes[l]

figure(figsize = (18,18))
subplot(121)
imshow(1-vesselness, cmap='gray')
title('1-vesselness', fontsize=18)
axis('off')
subplot(122)
imshow(ves_bin, cmap = 'gray')
title('after thresholding', fontsize=18)
axis('off')
show()

We now seem to be undersegmenting the cell membranes. We can finally combine this segmentation with the one obtained from thresholding.

In [ ]:
# Labels the connected components in the binary image
l,n = sndim.label(1-min_bin)

# Combines the thresholded image with the result of the vesselness filter 
sums = sndim.sum(1-ves_bin, l, np.unique(l))
final_segmentation = 1-(l == np.argmax(sums))

figure(figsize=(16,16))
subplot(221)
title('unsupervised segmentation', fontsize=18)
imshow(final_segmentation, cmap='gray')
axis('off')
subplot(222)
title('ground truth', fontsize=18)
imshow(ground_truth, cmap='gray')
axis('off')
subplot(223)
contours = skimage.segmentation.mark_boundaries(img,final_segmentation)
imshow(contours)
axis('off')
subplot(224)
imshow(contours_ground_truth)
axis('off')
show()

In [ ]:
# Calculates the pixel error (1-accuracy)
def pixel_error(auto_segmentation, ground_truth):
    return 1-((auto_segmentation==ground_truth).sum()/ground_truth.size)

print("pixel error: ", pixel_error(final_segmentation, ground_truth))

In [ ]:
# Puts everything together in a function
def segment_image_unsupervised(test_image):
    smoothed_img = sndim.gaussian_filter(test_image, sigma=2)
    th = skimage.filters.thresholding.threshold_minimum(smoothed_img)
    th_bin = smoothed_img > th
    # if the threshold minimum doens't work, use the Otsu method:
    if (th_bin.sum()/th_bin.size < 0.05) or (th_bin.sum()/th_bin.size > 0.95):
        th = skimage.filters.thresholding.threshold_otsu(smoothed_img)
        th_bin = smoothed_img > th
    perc_cells = th_bin.sum()/th_bin.size
    vesselness = skimage.filters.frangi(test_image, scale_range=(3,5))
    vesselness /= vesselness.max()
    ves_bin = vesselness <.1
    l, n = sndim.label(1-ves_bin)
    sizes = np.bincount(l.ravel())
    mask_sizes = sizes > 100
    mask_sizes[0] = 0
    ves_bin = 1-mask_sizes[l]
    l,n = sndim.label(1-th_bin)
    sums = sndim.sum(1-ves_bin, l, np.unique(l))
    final_segmentation = 1-(l == np.argmax(sums))
    return final_segmentation

# Evaluates the method on all validation images
inds = np.arange(n_images)
pixel_errors = np.zeros(inds.size)
for i,img_ind in enumerate(inds):
    img = data_validation[img_ind,...]
    ground_truth = labels_validation[img_ind,...]
    unsup_auto_segmentation = segment_image_unsupervised(img)
    pixel_errors[i] = pixel_error(unsup_auto_segmentation, ground_truth)
    
figure(figsize=(16,12))
bar(inds, pixel_errors)
show()
print("median pixel error: %0.2f" % (np.median(pixel_errors)))
print("avg pixel error: %0.2f +/- %0.2f" % (pixel_errors.mean(), pixel_errors.std()))


#  Can you do better?
 
 Move on to the [next notebook](challenge.ipynb)...